<a href="https://colab.research.google.com/github/qhungbui7/lenet-5/blob/main/lenet_5_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group 22



| Student ID  | Full name  |
|---|---|
| 20127508  |  Bùi Quốc Hùng |
|  20127521 |  Hồ Quang Khải |
|  20127011 |  Lê Tấn Đạt  |

## Work assignment

| Work | Assignee |
|---|---|
| Planning + Review + Environment Prepatation for MNIST + Report |  Bùi Quốc Hùng |
| Video | Hồ Quang Khải + Lê Tấn Đạt |
| Prepatation for Fashion MNIST |  Hồ Quang Khải |
| Implement a basic GPU Convolution kernel |  Hồ Quang Khải |
| Tiled shared memory convolution |Lê Tấn Đạt|
| Shared memory matrix multiplication and input matrix unrolling |Lê Tấn Đạt|
| Kernel fusion for unrolling and matrix-multiplication|Bùi Quốc Hùng|
| Weight matrix (kernel values) in constant memory |Lê Tấn Đạt|
| Tuning with restrict and loop unrolling|Bùi Quốc Hùng|
| Sweeping various parameters to find best values (block sizes, amount of thread coarsening) |Lê Tấn Đạt|
| Multiple kernel implementations for different layer sizes|Hồ Quang Khải|
| Input channel reduction: tree|Hồ Quang Khải|
| Input channel reduction: atomics |Bùi Quốc Hùng|
| Fixed point (FP16) arithmetic |Bùi Quốc Hùng|
| Using Streams to overlap computation with data transfer|Bùi Quốc Hùng|
| An advanced matrix multiplication algorithm |Hồ Quang Khải|




## How to run
- We recommend this method: Clone from our Github. The source code from this submission is identical to the one on our Github.
- About the data: https://drive.google.com/file/d/1KXvg7stExKuJvjlr-ScO4XQyi_iULIga/view?usp=drive_link (from https://www.kaggle.com/datasets/hojjatk/mnist-dataset)
- Where to put the data: Please create a folder, name it "MNIST" in Google Drive: My Drive -> MNIST

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/

/content


In [3]:
!rm -r /content/lenet-5

## NOTE
- Rememeber to specify the development branch when cloning something
- Change the execution file in CMakeLists.txt and execution below, for example I have written a new file called exp_newer_lenet.cc, then the execution file should be exp_newer_lenet
- CMake project only allow one int main() function in the project, then write the main file in the ./lenet-5/main/ and then copy them into the project to execute cmake

In [4]:
!git clone -b hung-dev  https://github.com/qhungbui7/lenet-5.git

Cloning into 'lenet-5'...
remote: Enumerating objects: 1830, done.
remote: Counting objects: 100% (1830/1830), done.
remote: Compressing objects: 100% (1462/1462), done.
remote: Total 1830 (delta 329), reused 1797 (delta 312), pack-reused 0
Receiving objects: 100% (1830/1830), 2.44 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (329/329), done.


In [5]:
%%writefile /content/lenet-5/main/exp_newer_lenet.cc

/*
 * CNN demo for MNIST dataset
 * Author: Kai Han (kaihana@163.com)
 * Details in https://github.com/iamhankai/mini-dnn-cpp
 * Copyright 2018 Kai Han
 */
#include <Eigen/Dense>
#include <algorithm>
#include <iostream>

#include "src/layer.h"
#include "src/layer/conv.h"
#include "src/layer/fully_connected.h"
#include "src/layer/ave_pooling.h"
#include "src/layer/max_pooling.h"
#include "src/layer/relu.h"
#include "src/layer/sigmoid.h"
#include "src/layer/softmax.h"
#include "src/loss.h"
#include "src/loss/mse_loss.h"
#include "src/loss/cross_entropy_loss.h"
#include "src/mnist.h"
#include "src/network.h"
#include "src/optimizer.h"
#include "src/optimizer/sgd.h"


int main() {
  // data
  MNIST dataset("../data/mnist/");
  dataset.read();
  std::cout << "Newer LeNet-5 implementation" << std::endl;
  int n_train = dataset.train_data.cols();
  int dim_in = dataset.train_data.rows();
  std::cout << "mnist train number: " << n_train << std::endl;
  std::cout << "mnist test number: " << dataset.test_labels.cols() << std::endl;
  // dnn

  // Conv(int channel_in, int height_in, int width_in, int channel_out,
  //      int height_kernel, int width_kernel, int stride = 1, int pad_w = 0,
  //      int pad_h = 0)


  // MaxPooling(int channel_in, int height_in, int width_in,
  //            int height_pool, int width_pool, int stride = 1) :

  // [(W−K+2P)/S]+1

  Network lenet5;
  Layer* conv1 = new Conv(1, 28, 28, 6, 5, 5, 1, 0, 0);
  Layer* relu1 = new ReLU;
  // (28 - 5 + 2 * 0) / 1 + 1 = 24

  Layer* pool2 = new MaxPooling(6, 24, 24, 2, 2, 1);
  // (24 - 2 + 2 * 0) / 1 + 1 = 23

  Layer* conv3 = new Conv(6, 23, 23, 16, 5, 5, 1, 0, 0);
  Layer* relu3 = new ReLU;
  // (23 - 5 + 2 * 0) / 1 + 1 = 19

  Layer* pool4 = new MaxPooling(16, 19, 19, 2, 2, 1);
  // (23 - 2 + 2 * 0) / 1 + 1 = 18

  Layer* fc6 = new FullyConnected(pool4->output_dim(), 120);
  Layer* relu6 = new ReLU;
  // 19 * 19 * 16 = 5776


  Layer* fc7 = new FullyConnected(120, 84);
  Layer* relu7 = new ReLU;

  Layer* fc8 = new FullyConnected(84, 10);
  Layer* softmax8 = new Softmax;

  lenet5.add_layer(conv1);
  lenet5.add_layer(relu1);

  lenet5.add_layer(pool2);

  lenet5.add_layer(conv3);
  lenet5.add_layer(relu3);

  lenet5.add_layer(pool4);

  lenet5.add_layer(fc6);
  lenet5.add_layer(relu6);

  lenet5.add_layer(fc7);
  lenet5.add_layer(relu7);

  lenet5.add_layer(fc8);
  lenet5.add_layer(softmax8);

  // loss
  Loss* loss = new CrossEntropy;
  lenet5.add_loss(loss);
  // train & test
  SGD opt(0.001, 5e-4, 0.9, true);
  // SGD opt(0.001);
  const int n_epoch = 5;
  const int batch_size = 128;
  for (int epoch = 0; epoch < n_epoch; epoch ++) {
    shuffle_data(dataset.train_data, dataset.train_labels);
    for (int start_idx = 0; start_idx < n_train; start_idx += batch_size) {
      int ith_batch = start_idx / batch_size;
      Matrix x_batch = dataset.train_data.block(0, start_idx, dim_in,
                                    std::min(batch_size, n_train - start_idx));
      Matrix label_batch = dataset.train_labels.block(0, start_idx, 1,
                                    std::min(batch_size, n_train - start_idx));
      Matrix target_batch = one_hot_encode(label_batch, 10);
      if (false && ith_batch % 10 == 1) {
        std::cout << ith_batch << "-th grad: " << std::endl;
        lenet5.check_gradient(x_batch, target_batch, 10);
      }
      lenet5.forward(x_batch);
      lenet5.backward(x_batch, target_batch);
      // display
      if (ith_batch % 50 == 0) {
        std::cout << ith_batch << "-th batch, loss: " << lenet5.get_loss()
        << std::endl;
      }
      // optimize
      lenet5.update(opt);
    }
    // test
    lenet5.forward(dataset.test_data);
    float acc = compute_accuracy(lenet5.output(), dataset.test_labels);
    std::cout << std::endl;
    std::cout << epoch + 1 << "-th epoch, test acc: " << acc << std::endl;
    std::cout << std::endl;
  }
  return 0;
}



Overwriting /content/lenet-5/main/exp_newer_lenet.cc


Copy file from ./lenet-5/main/ to project folder

In [6]:
!cp ./lenet-5/main/exp_newer_lenet.cc ./lenet-5/exp_newer_lenet.cc

In [7]:
%%writefile /content/lenet-5/CMakeLists.txt

cmake_minimum_required (VERSION 3.0)
project (myproject)
include_directories ("${PROJECT_SOURCE_DIR}/third_party/eigen")
# 查找当前目录下的所有源文件
# 并将名称保存到 DIR_SRCS 变量
aux_source_directory(. DIR_SRCS)
# 添加 math 子目录
add_subdirectory(src)
# 指定生成目标
add_executable(exp_newer_lenet ${DIR_SRCS})
# 添加链接库
target_link_libraries(exp_newer_lenet MiniDNNLib)

Overwriting /content/lenet-5/CMakeLists.txt


In [8]:
!mkdir ./lenet-5/data/
!mkdir ./lenet-5/data/mnist

Copy from Google Drive to this this folder, then unzip it into destination folder

In [9]:
!cp /content/drive/MyDrive/mnist/mnist.zip /content/mnist.zip

In [10]:
!unzip mnist.zip -d ./lenet-5/data/mnist

Archive:  mnist.zip
  inflating: ./lenet-5/data/mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-images.idx3-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-labels.idx1-ubyte  
  inflating: ./lenet-5/data/mnist/train-images-idx3-ubyte/train-images-idx3-ubyte  
  inflating: ./lenet-5/data/mnist/train-images.idx3-ubyte  
  inflating: ./lenet-5/data/mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte  
  inflating: ./lenet-5/data/mnist/train-labels.idx1-ubyte  


In [11]:
%cd ./lenet-5

/content/lenet-5


In [12]:
!ls ./data

mnist


In [13]:
%cd /content/lenet-5

/content/lenet-5


In [14]:
!mkdir /content/lenet-5/build
%cd /content/lenet-5/build
!cmake /content/lenet-5/
!make

/content/lenet-5/build
CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (1.4s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lenet-5/build
[  6%] Building CXX object src/CMakeFiles/MiniDNNLib

In [15]:
!ls

CMakeCache.txt	CMakeFiles  cmake_install.cmake  exp_newer_lenet  Makefile  src


In [ ]:
!./exp_newer_lenet

Newer LeNet-5 implementation
mnist train number: 60000
mnist test number: 10000
0-th batch, loss: 2.30286
50-th batch, loss: 2.30282
100-th batch, loss: 2.30024
150-th batch, loss: 2.29334
200-th batch, loss: 1.70971
250-th batch, loss: 0.438981
300-th batch, loss: 0.294733
350-th batch, loss: 0.248192
400-th batch, loss: 0.203345
450-th batch, loss: 0.155724

1-th epoch, test acc: 0.9401

0-th batch, loss: 0.280507
50-th batch, loss: 0.101838
100-th batch, loss: 0.0662025


In [ ]:
# !nvcc exp_newer_lenet.cu -o exp_newer_lenet

In [ ]:
# !./exp_newer_lenet

In [ ]:
# ! nvprof ./exp_newer_lenet